In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import net_utils as nu

In [2]:
### load cifar-10 dataset
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.cifar10.load_data()
onehot_train = nu.convert_to_onehot(y_train, 10)
onehot_test = nu.convert_to_onehot(y_test, 10)

In [ ]:
### visualize some of the images
fig, axes = plt.subplots(2,10,figsize=(20,5))
ctr = 0;
for i in range(2):
    for j in range(10):
        axes[i,j].imshow(np.squeeze(x_train[ctr]))
        axes[i,j].axes.get_xaxis().set_visible(False)
        axes[i,j].axes.get_yaxis().set_visible(False)
        axes[i,j].set_title(str(y_train[ctr]))
        ctr += 1
plt.show()

In [4]:
from basic_cnn import basic_cnn as basic_cnn
import tqdm
import time

class conv_net:
    def __init__(self, im_size, n_chan, n_class, learning_rate):
        self.im_size = im_size
        self.n_chan = n_chan
        self.n_class = n_class
        self.learning_rate = learning_rate

        self.inputs = tf.placeholder(tf.float32, (None, im_size, im_size, n_chan), name="inputs")
        self.targets = tf.placeholder(tf.float32, (None, n_class), name="targets")

        pred, cost = basic_cnn(self.inputs, self.targets)
        self.pred = pred
        self.cost = cost
        
        vars_to_minimize = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='conv_net')
        self.opt = tf.train.AdamOptimizer(self.learning_rate).minimize(cost, var_list=vars_to_minimize)
        
    def parameter_count(self, full=False):
        vars_to_minimize = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='conv_net')
        nu.parameter_count(vars_to_minimize, full = False)
        
    def train_on_batch(self, inputs, targets, mb_sz = 32):
        data_count = inputs.shape[0]

        for i in range(0, data_count, mb_sz):
            i_s = i
            i_e = np.minimum(data_count, i + mb_sz)
            sess_out = self.opt
            feed_dict={self.inputs: inputs[i_s:i_e], self.targets: targets[i_s:i_e]}
            sess.run(sess_out, feed_dict=feed_dict)
            time.sleep(0.01)
            
    def run_on_batch(self, inputs, targets, mb_sz = 32):
        data_count = inputs.shape[0]
        pred = np.zeros((data_count, self.n_class))
        cost = 0

        for i in range(0, data_count, mb_sz):
            i_s = i
            i_e = np.minimum(data_count, i + mb_sz)
            
            sess_out = [self.pred, self.cost]
            feed_dict={self.inputs: inputs[i_s:i_e], self.targets: targets[i_s:i_e]}
            results = sess.run(sess_out, feed_dict=feed_dict)
            pred[i_s:i_e] = results[0]
            cost += (results[1] / (i_e-i_s))
            time.sleep(0.01)
            
        return pred, cost

In [ ]:
### clear the graph
tf.reset_default_graph()
### build the network
net = conv_net(im_size = 32, n_chan = 3, n_class = 10, learning_rate = 1e-3)
net.parameter_count()
### initialze the session and variables
sess = tf.Session()
sess.run(tf.global_variables_initializer()) 

epochs = 20
for e in range(epochs):
    nu.shuffle_together(x_train, onehot_train)
    net.train_on_batch(x_train, onehot_train)
    _, trn_cost = net.run_on_batch(x_train, onehot_train)
    _, tst_cost = net.run_on_batch(x_test, onehot_test)
    
    print("Epoch: {}/{}...".format(e+1, epochs),
          "trn loss: {:.5f}".format(trn_cost),
          "tst loss: {:.5f}".format(tst_cost))

trainable parameters: 270270
Epoch: 1/20... trn loss: 69.44757 tst loss: 14.46718
Epoch: 2/20... trn loss: 60.96258 tst loss: 13.24848
Epoch: 3/20... trn loss: 52.94173 tst loss: 12.20011
Epoch: 4/20... trn loss: 48.66662 tst loss: 11.88518
Epoch: 5/20... trn loss: 43.87060 tst loss: 11.60608
Epoch: 6/20... trn loss: 42.03522 tst loss: 11.75096
Epoch: 7/20... trn loss: 41.09631 tst loss: 12.10887
